<a href="https://colab.research.google.com/github/valmirf/processamento_imagens/blob/master/Filtros/aula_04_opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Opencv - Aula 4

In [0]:
!git clone https://github.com/valmirf/processamento_imagens.git

Imports necessários

In [0]:
import cv2
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

## 1. TRANSFORMADA DE FOURIER

In [0]:
img = cv2.imread('processamento_imagens/imagens/alonecat.jpg',0)

rows, cols = img.shape
crow,ccol = rows//2 , cols//2 #pega ponto central da imagem

dft = cv2.dft(np.float32(img),flags = cv2.DFT_COMPLEX_OUTPUT) #transformada discreta de fourier
dft_shift = np.fft.fftshift(dft) #mudança do ponto DC

magnitude_spectrum = 20*np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1])) #logarítimo para melhor visualização

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
ax = axes.ravel()
ax[0].imshow(img, cmap='gray')
ax[0].set_title('Imagem Entrada')
ax[0].set_axis_off()

ax[1].imshow(magnitude_spectrum, cmap='gray')
ax[1].set_title('Magnitude Spectrum')
ax[1].set_axis_off()

### 1.1 TRANSFORMADA INVERSA

In [0]:
#retoma o ponto central para o canto da imagem
f_ishift = np.fft.ifftshift(dft_shift)
#transformada inversa
img_back = cv2.idft(f_ishift)
img_back = cv2.magnitude(img_back[:,:,0],img_back[:,:,1])

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
ax = axes.ravel()
ax[0].imshow(magnitude_spectrum, cmap='gray')
ax[0].set_title('Magnitude Spectrum')
ax[0].set_axis_off()


ax[1].imshow(img_back, cmap='gray')
ax[1].set_title('Imagem de Saída')
ax[1].set_axis_off()


## 2. Filtragem no domínio da frequência

###2.1 Filtro Simples (Passa-Baixa)


In [0]:
# create a mask first, center square is 1, remaining all zeros
mask = np.zeros((rows,cols,2))
mask[crow-30:crow+30, ccol-30:ccol+30] = 1

# apply mask and inverse DFT
fshift = dft_shift*mask
f_ishift = np.fft.ifftshift(fshift)
img_back = cv2.idft(f_ishift)
img_back = cv2.magnitude(img_back[:,:,0],img_back[:,:,1])

fig, axes = plt.subplots(1, 3, figsize=(10, 5))
ax = axes.ravel()
ax[0].imshow(magnitude_spectrum, cmap='gray')
ax[0].set_title('Magnitude Spectrum')
ax[0].set_axis_off()

ax[1].imshow(mask[:,:,0], cmap='gray')
ax[1].set_title('Máscara')
ax[1].set_axis_off()

ax[2].imshow(img_back, cmap='gray')
ax[2].set_title('Imagem de Saída')
ax[2].set_axis_off()


###2.2 Filtro Simples (Passa-Alta)

In [0]:
#inverte o filtro
mask = 1 - mask #filtro passa-alta

# apply mask and inverse DFT
fshift = dft_shift*mask
f_ishift = np.fft.ifftshift(fshift)
img_back = cv2.idft(f_ishift)
img_back = cv2.magnitude(img_back[:,:,0],img_back[:,:,1])

fig, axes = plt.subplots(1, 3, figsize=(10, 5))
ax = axes.ravel()
ax[0].imshow(magnitude_spectrum, cmap='gray')
ax[0].set_title('Magnitude Spectrum')
ax[0].set_axis_off()

ax[1].imshow(mask[:,:,0], cmap='gray')
ax[1].set_title('Máscara')
ax[1].set_axis_off()

ax[2].imshow(img_back, cmap='gray')
ax[2].set_title('Imagem de Saída')
ax[2].set_axis_off()


### 2.1 Gaussiano (Passa-Baixa)

In [0]:
#imagem quadrada pra facilitar cálculos
imgg = cv2.resize(img,(rows,rows)) 
print(imgg.shape)

rows, cols = imgg.shape

# inicializa a máscara
mask = np.zeros((rows,rows,2))
print(mask.shape)

# cria o filtro gaussiano
kernel = cv2.getGaussianKernel(rows,sigma=5)
kernel = kernel * kernel.T

mask[:,:,0] =kernel
mask[:,:,1] =kernel

#trasformada inversa
dftg = cv2.dft(np.float32(imgg),flags = cv2.DFT_COMPLEX_OUTPUT) #transformada discreta de fourier
dft_shiftg = np.fft.fftshift(dftg) #mudança do ponto DC

magnitude_spectrumg = 20*np.log(cv2.magnitude(dft_shiftg[:,:,0],dft_shiftg[:,:,1])) #logarítimo para melhor visualização

# aplica a filtragem gaussiana
fshift = dft_shiftg*mask
# transformada inversa
f_ishift = np.fft.ifftshift(fshift)
img_back = cv2.idft(f_ishift)
img_back = cv2.magnitude(img_back[:,:,0],img_back[:,:,1])

fig, axes = plt.subplots(1, 3, figsize=(10, 5))
ax = axes.ravel()
ax[0].imshow(magnitude_spectrumg, cmap='gray')
ax[0].set_title('Magnitude Spectrum')
ax[0].set_axis_off()

ax[1].imshow(mask[:,:,0], cmap='gray')
ax[1].set_title('Filtro')
ax[1].set_axis_off()

ax[2].imshow(img_back, cmap='gray')
ax[2].set_title('Imagem de Saída')
ax[2].set_axis_off()

## Exercícios

1. Construa um filtro passa-alta Gaussiano da imagem anterior

2. Construa a máscara do filtro butterwoth. Depois aplique o filtro passa-baixa e passa-alta desse filtro.